<a href="https://colab.research.google.com/github/squadOito/soulcodead2/blob/joseaureliok%2Fnotebook/notebooks/notebook_iea_ponto_veiculos_pyspark_bruto.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Projeto Final**
Escola: SoulCode Academy

Curso: Bootcamp Analista de Dados - Martech - AD2

**Equipe 08**

**Alunos: Adriano Kim, José Aurelio, Marcos Paulo, Paulo Vitorino, Renato e Wesley**

Professores: Douglas Ribeiro, Franciane Rodrigues e Jonathas Carneiro

## Preparação de Ambiente
Instalações e importações das bibliotecas necessárias para o processo de ETL.

In [ ]:
# Instalaçao Bibliotecas
!pip install gcsfs -q

In [ ]:
# Importando Bibliotecas
import os
import pandas as pd
import numpy as np

from google.colab import data_table
from google.cloud import storage
from google.colab import drive
from google.colab import auth
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from oauth2client.client import GoogleCredentials

In [ ]:
# Ignorando alguns alertas desnecessários
import warnings
warnings.filterwarnings("ignore")

In [ ]:
# Configuração da quantidade de colunas para aparecer em um DataFrame
pd.set_option('display.max_columns',100)

### Configuração PySpark

In [ ]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -N -q http://archive.apache.org/dist/spark/spark-3.1.1/spark-3.1.1-bin-hadoop3.2.tgz
!tar xf spark-3.1.1-bin-hadoop3.2.tgz
!pip install -q findspark

In [ ]:
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.1-bin-hadoop3.2"

In [ ]:
import findspark
findspark.init()

from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()

from pyspark.sql.functions import regexp_replace
spark.conf.set("spark.sql.repl.eagerEval.enabled", True)

import pyspark.sql.functions as F
from pyspark.sql.types import *

spark

### Compartilhamento Chave GDrive

In [ ]:
# Cria compartilhamento com Google Drive
drive.mount('/content/drive', force_remount=True)

# Arquivo a ser acessado na pasta compartilhada

target = 'projeto-final-ad2-e8-ae566c3a2c2b.json'

# Caminho completo da pasta compartilhada
folder = '/content/drive/MyDrive/Classroom/AD2 - Analista de Dados/ProjetoFinal'

# Acesso ao arquivo no colab
serviceAccount = os.path.join(folder, target)

Mounted at /content/drive


### Conexão Google Cloud

In [ ]:
# Conexão com a conta do Google Cloud
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = serviceAccount

In [ ]:
# Conexão com a bucket do Google Cloud
client = storage.Client()
bucket = client.get_bucket('projeto-final-ad2-e8')

## Funções

In [ ]:
# função de leitura no Spark
def spark_read(path):
  try:

    try:
      # leitura csv com pandas
      read = pd.read_excel(path)
      # conversão em df pandas em df pyspark
      df = spark.createDataFrame(read)
      # êxito da função retornando o dataframe
      return df

    except:
      # leitura csv com pandas
      read = pd.read_excel(path)

      # tratamento de tipo de dado do df pandas para conversão em df pyspark
      for n in read.columns:
        if read[f'{n}'].dtype == object:
          read[f'{n}'] = read[f'{n}'].astype(str)
        elif read[f'{n}'].dtype == bool:
          read[f'{n}'] = read[f'{n}'].astype(bool)
        else:
          read[f'{n}'] = read[f'{n}'].astype(np.double)

      # conversão df pandas em df pyspark
      df = spark.createDataFrame(read)
      # êxito da função retornando o dataframe
      return df

  except:
    # indicação de erro na leitura do dataframe
    print(f'Falha na leitura do DataFrame: {path}!')

In [ ]:
# Função contagem distintos por atributos
def listDistinct(dataframe):
  '''
  Executa a contagem de valores distintos por atributo!
  dataframe = nome do dataframe, tipo string
  '''
  lista_distintos = [F.approx_count_distinct(col).alias(col) for col in dataframe.columns]
  distintos = dataframe.agg(*lista_distintos)
  data_table.enable_dataframe_formatter()
  distintos.show(truncate=False)

In [ ]:
# Função contagem distintos por registro
def countDistinct(dataframe):
  '''
  Exibe valores distintos por registro!
  dataframe = nome do dataframe, tipo string
  '''
  for coluna in dataframe.columns:
    contagem = dataframe.groupby(coluna).count().orderBy(F.col('count').desc())
    print('==*'*70)
    print(coluna)
    print('==*'*70)
    contagem.show(n=50, truncate=False)

In [ ]:
# Função contagem nulos
def fnull(dataframe):
  '''
  Exibe valores nulos por atributo!
  dataframe = nome do dataframe, tipo string
  '''
  lista_nulos = [F.sum(F.when(F.col(c).isNull(), 1).otherwise(0)).alias(c) for c in dataframe.columns]
  nulos = dataframe.agg(*lista_nulos)
  nulos.show(truncate = False)

In [ ]:
# Função validação
def validar(dataframe, tabela):
  '''
  tabela = nome da tabela
  dataframe = nome do dataframe, tipo string
  '''
  # estabelecendo laço for para quantificar valores nulos de cada coluna
  nl = [F.sum(F.when(F.col(c).isNull(), 1).otherwise(0)).alias(c) for c in dataframe.columns]

  # somando valores nulos encontrados para cada coluna, obtidos através de dicionários e adicionados em listas
  nulos = sum(list(((dataframe.agg(*nl).collect()[0]).asDict()).values()))

  schema1 = dataframe.schema == schemaBR1
  schema2 = dataframe.schema == schemaBR2

  # validando Schema e valores nulos do dataframe tratado
  if schema1 == True and nulos == 0:
    print( f'Tabela {tabela} VALIDADA!')
  elif schema2 == True and nulos == 0:
    print( f'Tabela {tabela} VALIDADA!')
  else:
    print( f'Tabela {tabela} NÃO validada:\n -> Schema {schema1} e {schema2};\n -> Nulos: {nulos}!')

## Extração
A primeira etapa da ETL é a extração dos dados de sua fonte original. Dependendo do tipo de dados e da fonte, você pode precisar de diferentes ferramentas e técnicas para extrair os dados.

In [ ]:
# Caminho do arquivo entrada
pathPontoRecarga = 'gs://projeto-final-ad2-e8/dados/brutos/excel/iea_ponto_recarga_bruto.xlsx'
pathVeiculos = 'gs://projeto-final-ad2-e8/dados/brutos/excel/iea_veiculos_bruto data.xlsx'

# Caminho do arquivo saída
outpathPontoRecarga = 'gs://projeto-final-ad2-e8/dados/tratados/iea_ponto_recarga_tratado.csv'
outpathVeiculos = 'gs://projeto-final-ad2-e8/dados/tratados/iea_veiculos_tratado.csv'

### Read Pyspark

In [ ]:
# Leitura do conjunto de dados iea_ponto_recarga
dfp = spark_read(pathPontoRecarga)

In [ ]:
# Leitura do conjunto de dados iea_veiculos
dfv = spark_read(pathVeiculos)

### Pré Análise

In [ ]:
# Visualização geral Ponto Recarga
print('Ponto Recarga')
dfp.show(n=50, truncate=False)

Ponto Recarga
+---------+----------+------------------+----+-----------------------+----+---------------+-----------------+
|region   |category  |parameter         |mode|powertrain             |year|unit           |value            |
+---------+----------+------------------+----+-----------------------+----+---------------+-----------------+
|Australia|Historical|EV charging points|EV  |Publicly available fast|2017|charging points|40               |
|Australia|Historical|EV charging points|EV  |Publicly available slow|2017|charging points|440              |
|Australia|Historical|EV charging points|EV  |Publicly available fast|2018|charging points|61               |
|Australia|Historical|EV charging points|EV  |Publicly available slow|2018|charging points|670              |
|Australia|Historical|EV charging points|EV  |Publicly available fast|2019|charging points|250              |
|Australia|Historical|EV charging points|EV  |Publicly available slow|2019|charging points|1700           

In [ ]:
# Visualização geral Veículos
print('Veiculos')
dfv.show(n=50, truncate=False)

Veiculos
+---------+----------+------------------+----+-----------------------+------+---------------+-----------------+
|region   |category  |parameter         |mode|powertrain             |year  |unit           |value            |
+---------+----------+------------------+----+-----------------------+------+---------------+-----------------+
|Australia|Historical|EV sales          |Cars|BEV                    |2011.0|Vehicles       |49               |
|Australia|Historical|EV stock          |Cars|BEV                    |2011.0|Vehicles       |49               |
|Australia|Historical|EV sales share    |Cars|EV                     |2011.0|percent        |6500000134110450 |
|Australia|Historical|EV stock share    |Cars|EV                     |2011.0|percent        |4600000102072950 |
|Australia|Historical|EV stock share    |Cars|EV                     |2012.0|percent        |279999990016222  |
|Australia|Historical|EV sales share    |Cars|EV                     |2012.0|percent        |29

In [ ]:
# Tamanho total de (linhas , colunas)
describep = dfp.describe()
describev = dfv.describe()

print('Ponto Recarga')
print(describep)
print('Veiculos')
print('-'*100)
print(describev)

Ponto Recarga
+-------+--------------+----------+------------------+----+--------------------+-----------------+---------------+--------------------+
|summary|        region|  category|         parameter|mode|          powertrain|             year|           unit|               value|
+-------+--------------+----------+------------------+----+--------------------+-----------------+---------------+--------------------+
|  count|           604|       604|               604| 604|                 604|              604|            604|                 604|
|   mean|          null|      null|              null|null|                null|2017.546357615894|           null|5.703642471083345E14|
| stddev|          null|      null|              null|null|                null|3.126864813215356|           null|2.275968481157552E15|
|    min|     Australia|Historical|EV charging points|  EV|Publicly availabl...|             2010|charging points|                   1|
|    max|United Kingdom|Historical

In [ ]:
# Todos os tipos de dados presentes
tipop = dfp.dtypes
tipov = dfv.dtypes

print('Ponto Recarga')
print(tipop)
print('\n')
print('Veiculos')
print('-'*100)
print(tipov)

Ponto Recarga
[('region', 'string'), ('category', 'string'), ('parameter', 'string'), ('mode', 'string'), ('powertrain', 'string'), ('year', 'bigint'), ('unit', 'string'), ('value', 'bigint')]


Veiculos
----------------------------------------------------------------------------------------------------
[('region', 'string'), ('category', 'string'), ('parameter', 'string'), ('mode', 'string'), ('powertrain', 'string'), ('year', 'double'), ('unit', 'string'), ('value', 'string')]


### Limpeza de Dados
Remover dados duplicados, corrigir erros de digitação, tratar dados inconsistentes etc.

#### Valores distintos

In [ ]:
# Valores distintos por atributo
print('Ponto Recarga')
atrp = listDistinct(dfp)

print('Veiculos')
atrv = listDistinct(dfv)

Ponto Recarga
+------+--------+---------+----+----------+----+----+-----+
|region|category|parameter|mode|powertrain|year|unit|value|
+------+--------+---------+----+----------+----+----+-----+
|33    |1       |1        |1   |2         |13  |1   |231  |
+------+--------+---------+----+----------+----+----+-----+

Veiculos
+------+--------+---------+----+----------+----+----+-----+
|region|category|parameter|mode|powertrain|year|unit|value|
+------+--------+---------+----+----------+----+----+-----+
|37    |3       |8        |5   |5         |15  |6   |997  |
+------+--------+---------+----+----------+----+----+-----+



In [ ]:
# Aplicando função para contar valores distintos por registro
print('Ponto Recarga')
countDistinct(dfp)

Ponto Recarga
==*==*==*==*==*==*==*==*==*==*==*==*==*==*==*==*==*==*==*==*==*==*==*==*==*==*==*==*==*==*==*==*==*==*==*==*==*==*==*==*==*==*==*==*==*==*==*==*==*==*==*==*==*==*==*==*==*==*==*==*==*==*==*==*==*==*==*==*==*==*
region
==*==*==*==*==*==*==*==*==*==*==*==*==*==*==*==*==*==*==*==*==*==*==*==*==*==*==*==*==*==*==*==*==*==*==*==*==*==*==*==*==*==*==*==*==*==*==*==*==*==*==*==*==*==*==*==*==*==*==*==*==*==*==*==*==*==*==*==*==*==*
+--------------+-----+
|region        |count|
+--------------+-----+
|Norway        |25   |
|Chile         |24   |
|Portugal      |24   |
|Austria       |24   |
|Netherlands   |24   |
|United Kingdom|23   |
|Italy         |22   |
|Switzerland   |22   |
|Canada        |22   |
|Japan         |22   |
|France        |22   |
|Denmark       |22   |
|Germany       |22   |
|Spain         |21   |
|USA           |21   |
|Belgium       |20   |
|Finland       |20   |
|Greece        |19   |
|Poland        |18   |
|China         |18   |
|Turkiye       |17   |
|Icel

In [ ]:
# Aplicando função para contar valores distintos por registro
print('Veiculos')
countDistinct(dfv)

Veiculos
==*==*==*==*==*==*==*==*==*==*==*==*==*==*==*==*==*==*==*==*==*==*==*==*==*==*==*==*==*==*==*==*==*==*==*==*==*==*==*==*==*==*==*==*==*==*==*==*==*==*==*==*==*==*==*==*==*==*==*==*==*==*==*==*==*==*==*==*==*==*
region
==*==*==*==*==*==*==*==*==*==*==*==*==*==*==*==*==*==*==*==*==*==*==*==*==*==*==*==*==*==*==*==*==*==*==*==*==*==*==*==*==*==*==*==*==*==*==*==*==*==*==*==*==*==*==*==*==*==*==*==*==*==*==*==*==*==*==*==*==*==*
+-----------------+-----+
|region           |count|
+-----------------+-----+
|China            |948  |
|World            |942  |
|Europe           |931  |
|Rest of the world|779  |
|India            |510  |
|USA              |374  |
|EU27             |284  |
|United Kingdom   |283  |
|France           |270  |
|Denmark          |262  |
|Netherlands      |259  |
|Belgium          |255  |
|Austria          |231  |
|Other Europe     |223  |
|Sweden           |220  |
|Germany          |219  |
|Norway           |217  |
|Japan            |208  |
|Italy          

#### Valores Nulos

In [ ]:
# Verificar se há valores nulos/ausentes
print('Ponto Recarga')
fnull(dfp)

print('Veiculos')
fnull(dfv)

Ponto Recarga
+------+--------+---------+----+----------+----+----+-----+
|region|category|parameter|mode|powertrain|year|unit|value|
+------+--------+---------+----+----------+----+----+-----+
|0     |0       |0        |0   |0         |0   |0   |0    |
+------+--------+---------+----+----------+----+----+-----+

Veiculos
+------+--------+---------+----+----------+----+----+-----+
|region|category|parameter|mode|powertrain|year|unit|value|
+------+--------+---------+----+----------+----+----+-----+
|0     |0       |0        |0   |0         |0   |0   |0    |
+------+--------+---------+----+----------+----+----+-----+



### Normalização de Dados
Colocar os dados em um formato padronizado para facilitar a análise.

#### Renomeando

In [ ]:
# Backup dataframes
pbr = dfp
vbr = dfv

print('Ponto Recarga')
pbr.show(truncate=False)

print('Veiculos')
vbr.show(truncate=False)

Ponto Recarga
+---------+----------+------------------+----+-----------------------+----+---------------+-----------------+
|region   |category  |parameter         |mode|powertrain             |year|unit           |value            |
+---------+----------+------------------+----+-----------------------+----+---------------+-----------------+
|Australia|Historical|EV charging points|EV  |Publicly available fast|2017|charging points|40               |
|Australia|Historical|EV charging points|EV  |Publicly available slow|2017|charging points|440              |
|Australia|Historical|EV charging points|EV  |Publicly available fast|2018|charging points|61               |
|Australia|Historical|EV charging points|EV  |Publicly available slow|2018|charging points|670              |
|Australia|Historical|EV charging points|EV  |Publicly available fast|2019|charging points|250              |
|Australia|Historical|EV charging points|EV  |Publicly available slow|2019|charging points|1700           

In [ ]:
# Renomeando registros do dataframe
traducao = {
    'Publicly available slow': 'LENTO',
    'Publicly available fast': 'RAPIDO',
    "Sweden": "SUECIA",
    "Germany": "ALEMANHA",
    "France": "FRANCA",
    "Greece": "GRECIA",
    "Belgium": "BELGICA",
    "Finland": "FINLANDIA",
    "China": "CHINA",
    "India": "INDIA",
    "Chile": "CHILE",
    "Italy": "ITALIA",
    "Norway": "NORUEGA",
    "Spain": "ESPANHA",
    "Denmark": "DINAMARCA",
    "Thailand": "TAILANDIA",
    "Iceland": "ISLANDIA",
    "Israel": "ISRAEL",
    "USA": "ESTADOS UNIDOS",
    "Mexico": "MEXICO",
    "Indonesia": "INDONESIA",
    "Switzerland": "SUIÇA",
    "Turkiye": "TURQUIA",
    "Canada": "CANADA",
    "Brazil": "BRASIL",
    "Japan": "JAPAO",
    "New Zealand": "NOVA ZELANDIA",
    "Poland": "POLONIA",
    "Portugal": "PORTUGAL",
    "Australia": "AUSTRALIA",
    "Austria": "AUSTRIA",
    "Korea": "COREIA",
    "South Africa": "AFRICA DO SUL",
    "United Kingdom": "REINO UNIDO",
    "Netherlands": "PAISES BAIXOS",
    "World" : "GLOBAL",
    "Rest of the world": "OUTROS GLOBAL",
    "Europe": "EUROPA",
    "Other Europe": "OUTROS EUROPA",
    "EU27": "UNIAO EUROPEIA",
    "Historical": "HISTORICO",
    "Projection-STEPS": "CENARIO POLITICA DECLARADA",
    "Projection-APS": "CENARIO POLITICA ANUNCIADA",
    "charging points": "PONTOS DE CARREGAMENTO",
    "EV stock share": "PARTICIPACAO NO ESTOQUE DE VEICULOS ELETRICOS",
    "Electricity demand": "DEMANDA ENERGIA ELETRICA",
    "EV sales": "VENDAS",
    "EV sales share": "PARTICIPACAO NAS VENDAS DE VEICULOS ELETRICOS",
    "Oil displacement Mbd": "DESLOCAMENTO DE OLEO MBD",
    "EV stock" : "ESTOQUE",
    "Vehicles" : "VEICULOS",
    "percent" : "PERCENTUAL",
    "Milion barrels per day" : "MILHOES BARRIS POR DIA",
    "Oil displacement, million lge" : "DESLOCAMENTO DE OLEO EM MILHOES LGE",
    "GWh" : "GIGAWATTS HORA",
    "EV charging points": "PONTOS DE CARREGAMENTO VEICULAR",
    "EV": "VEICULOS ELETRICOS",
    "PHEV": "VEICULO ELETRICO HIBRIDO",
    "BEV": "VEICULO ELETRICO EXCLUSIVO",
    "Buses": "ONIBUS",
    "Cars": "CARROS",
    "Trucks": "CAMINHOES",
    "Vans": "UTILITARIOS"
}

# LGE = líquido gerador de espuma
# Fonte: https://www.iea.org/data-and-statistics/data-tools/global-ev-data-explorer

In [ ]:
# Renomeando registros
print('Ponto Recarga')
for i in pbr.columns:
    pbr = pbr.withColumn(i, F.coalesce(*(F.when(F.col(i) == value, F.lit(traducao[value])) for value in traducao.keys()), F.col(i)))

Ponto Recarga


In [ ]:
# Renomeando registros
print('Veiculo')
for i in vbr.columns:
    vbr = vbr.withColumn(i, F.coalesce(*(F.when(F.col(i) == value, F.lit(traducao[value])) for value in traducao.keys()), F.col(i)))

Veiculo


In [ ]:
# tradução das colunas do dataframe
pbr = pbr.withColumnRenamed("region", "pais")\
         .withColumnRenamed("category", "categoria")\
         .withColumnRenamed("parameter", "parametro")\
         .withColumnRenamed("mode", "veiculo")\
         .withColumnRenamed("powertrain", "carregamento")\
         .withColumnRenamed("unit", "unidade")\
         .withColumn("valor", F.col("value").cast("long"))\
         .withColumn("ano", F.col("year").cast("int"))

vbr = vbr.withColumnRenamed("region", "pais")\
         .withColumnRenamed("category", "categoria")\
         .withColumnRenamed("parameter", "parametro")\
         .withColumnRenamed("mode", "veiculo")\
         .withColumnRenamed("powertrain", "carregamento")\
         .withColumnRenamed("unit", "unidade")\
         .withColumn("valor", F.col("value"))\
         .withColumn("ano", F.col("year").cast("int"))

In [ ]:
# Tipos de dados
print('Ponto Recarga')
pbr.dtypes

Ponto Recarga


[('pais', 'string'),
 ('categoria', 'string'),
 ('parametro', 'string'),
 ('veiculo', 'string'),
 ('carregamento', 'string'),
 ('year', 'string'),
 ('unidade', 'string'),
 ('value', 'string'),
 ('valor', 'bigint'),
 ('ano', 'int')]

In [ ]:
# Tipos de daosd
print('Veiculos')
vbr.dtypes

Veiculos


[('pais', 'string'),
 ('categoria', 'string'),
 ('parametro', 'string'),
 ('veiculo', 'string'),
 ('carregamento', 'string'),
 ('year', 'string'),
 ('unidade', 'string'),
 ('value', 'string'),
 ('valor', 'string'),
 ('ano', 'int')]

In [ ]:
# Eliminação de colunas redundantes
print('Ponto Recarga')
pbr = pbr.drop('year', 'value')
print(pbr.columns)

print('\n')

print('Veiculos')
vbr = vbr.drop('year', 'value')
print(vbr.columns)

Ponto Recarga
['pais', 'categoria', 'parametro', 'veiculo', 'carregamento', 'unidade', 'valor', 'ano']


Veiculos
['pais', 'categoria', 'parametro', 'veiculo', 'carregamento', 'unidade', 'valor', 'ano']


# **Qualidade e integridade dos dados**

### **Validação da estrutura dos dados (schema)**

In [ ]:
# Definindo o esquema de validação e o examinando
schemaBR1 = StructType([ StructField("pais", StringType()),
                        StructField("categoria", StringType()),
                        StructField("parametro", StringType()),
                        StructField("veiculo", StringType()),
                        StructField("carregamento", StringType()),
                        StructField("unidade", StringType()),
                        StructField("valor", LongType()),
                        StructField("ano", IntegerType())
                      ])

schemaBR2 = StructType([ StructField("pais", StringType()),
                        StructField("categoria", StringType()),
                        StructField("parametro", StringType()),
                        StructField("veiculo", StringType()),
                        StructField("carregamento", StringType()),
                        StructField("unidade", StringType()),
                        StructField("valor", StringType()),
                        StructField("ano", IntegerType())
                      ])

In [ ]:
# Verificando valores nulos remanescentes
pbr.where( F.col('valor').isNull()).show(n=75, truncate=False)

+----+---------+---------+-------+------------+-------+-----+---+
|pais|categoria|parametro|veiculo|carregamento|unidade|valor|ano|
+----+---------+---------+-------+------------+-------+-----+---+
+----+---------+---------+-------+------------+-------+-----+---+



In [ ]:
# Verificando valores nulos remanescentes
vbr.where( F.col('valor').isNull()).show(n=100, truncate=False)

+----+---------+---------+-------+------------+-------+-----+---+
|pais|categoria|parametro|veiculo|carregamento|unidade|valor|ano|
+----+---------+---------+-------+------------+-------+-----+---+
+----+---------+---------+-------+------------+-------+-----+---+



In [ ]:
# Aplicando a função para validação do Schema e inexistência de registros nulos
validar(pbr, 'iea_ponto_revenda')

Tabela iea_ponto_revenda VALIDADA!


In [ ]:
# Aplicando a função para validação do Schema e inexistência de registros nulos
validar(vbr, 'iea_veiculos')

Tabela iea_veiculos VALIDADA!


## **Colunas Persistidas**

================================================================================
String
================================================================================
* pais
* categoria
* parametro
* veiculo
* carregamento
* unidade

================================================================================
BigInt
================================================================================
* valor

================================================================================
Int
================================================================================
* ano


## Copia de segurança do tratamento - Backup

In [ ]:
# Backup para analise
df_tratado_ponto = pbr
df_tratado_veiculo = vbr

# Load
Depois que os dados são extraídos e transformados adequadamente, eles estarão prontos para as análise, mas antes disso eles precisam ser carregados em um local de armazenamento adequado. Podendo ser um banco de dados SQL ou NoSQL, um sistema de armazenamento em nuvem, e para o caso de disponibilizar o projeto publicamente é ideal que ele seja colocar em uma pasta de datasets, diferenciando o arquivo bruto e o tratado.

In [ ]:
# convertendo em um dataframe Pandas
gcsp = pbr.toPandas()
gcsv = vbr.toPandas()

# carregamento no GCS
gcsp.to_csv(outpathPontoRecarga, index=False)
gcsv.to_csv(outpathVeiculos, index=False)

# encerrando o SparkSession
spark.stop()